# Bert Inference

In [2]:
!pip install -q "sagemaker==2.198" "transformers==4.26.0" "datasets==2.9" "torch==1.13.1" "tqdm"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
awscli 1.29.63 requires botocore==1.31.63, but you have botocore 1.33.11 which is incompatible.
awscli 1.29.63 requires s3transfer<0.8.0,>=0.7.0, but you have s3transfer 0.8.2 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.16.1 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.1 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import sagemaker
import boto3
import pandas as pd
import json
import csv
from tqdm import tqdm
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner
)
from ast import literal_eval

from sagemaker.huggingface import HuggingFace, HuggingFaceModel
from sagemaker.s3 import S3Uploader, S3Downloader, s3_path_join
from datasets import load_dataset
from transformers import AutoTokenizer, get_constant_schedule, get_constant_schedule_with_warmup
import s3fs
import re
#from utils import summarize_hpo_results

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
#Set up sagemaker session
sess = sagemaker.Session()
sagemaker_session_bucket='sagemaker-studio-ai-lab-3'
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::956796298028:role/service-role/AmazonSageMaker-ExecutionRole-20211005T133138
sagemaker bucket: sagemaker-studio-ai-lab-3
sagemaker session region: us-east-1


In [6]:
huggingface_model = HuggingFaceModel(
    model_data=r"s3://sagemaker-studio-ai-lab-3/samples/datasets/tokenized_reviews_v4/bert_model/LA-bert-base-60k-v2-2023-12-08-21-58-47-098/output/model.tar.gz",
        #old model: "s3://sagemaker-studio-ai-lab-3/samples/datasets/tokenized_reviews_v2/bert_model/LA-bert-45k-v1-2023-12-02-03-29-43-447/output/model.tar.gz",
    role=role, # iam role with permissions to create an Endpoint
    transformers_version="4.26.0",
    pytorch_version="1.13.1", 
    py_version='py39'
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


BatchTransform to gather predictions from BERT
    must be in jsonl file type

In [7]:
#load all real data (only r_text, review_id, r_star column) from s3 (6.3 mil), then clean, then transform to jsonl, then upload clean version to s3 (one time use)

#loading data
dfs = []
for load_num in tqdm(range(0,70)):
    df = pd.read_json(f's3://sagemaker-studio-ai-lab-3/final-data/data_{load_num}.json') 
    dfs.append(df)
reviews = pd.concat (dfs, ignore_index = True)

#cleaning text comments by tokenizing text
REPLACE_BY_SPACE_RE = re.compile('--[!/(){}\[\]\n,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
def clean_text(text):
    """
        text: a string
        return: modified initial string
    """
    text = text.lower() 
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub(' ', text) 
    return text
reviews['r_text'] = reviews['r_text'].apply(clean_text)

#convert text to jsonl format and uploads to s3
reviews_jsonl = f"reviews_text.jsonl"
with open(reviews_jsonl, "w+") as outfile:
    for value in tqdm(reviews['r_text']):
        json.dump({"inputs": value, "parameters":{"truncation":True, "padding":'max_length',  "max_length": 512}}, outfile) #used to be 128
        outfile.write('\n')
s3_file_uri = S3Uploader.upload(reviews_jsonl,"s3://sagemaker-studio-ai-lab-3/samples/datasets/inference/batch_transform/bert60k_input") 

100%|██████████| 6291252/6291252 [02:30<00:00, 41914.88it/s]


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [ ]:
#start batch job wit jsonl files
batch_job = huggingface_model.transformer(
    instance_count=1,
    instance_type='ml.g4dn.xlarge', #ml.p3.2xlarge
    output_path="s3://sagemaker-studio-ai-lab-3/samples/datasets/inference/batch_transform/bert60k_output",
    strategy='SingleRecord')

batch_job.transform(
    data=s3_file_uri, 
    content_type='application/json',    
    split_type='Line')

In [5]:
# creating s3 uri for result file -> input file + .out
output_file = f"reviews_text.jsonl.out"
output_path = s3_path_join("s3://sagemaker-studio-ai-lab-3/samples/datasets/inference/batch_transform/bert60k_output",output_file)

# download file
S3Downloader.download(output_path,'.')

batch_transform_result = []
with open(output_file) as f:
    for line in f:
        # converts jsonline array to normal array
        line = "[" + line.replace("[","").replace("]",",") + "]"
        batch_transform_result = literal_eval(line) 

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [6]:
dfs = []
for load_num in tqdm(range(0,70)):
    df = pd.read_json(f's3://sagemaker-studio-ai-lab-3/final-data/data_{load_num}.json') 
    dfs.append(df)
reviews = pd.concat (dfs, ignore_index = True)

100%|██████████| 70/70 [14:27<00:00, 12.40s/it]


In [7]:
print(len(batch_transform_result))
print(len(reviews))

6291252
6291252


In [8]:
reviews = reviews.assign(bert_pred = batch_transform_result)

#change all 0 indexed labels into normal labels again
for i in tqdm(range(len(reviews.bert_pred))):
    reviews.bert_pred[i]['label'] = reviews.bert_pred[i]['label'][:-1] + str(int(reviews.bert_pred[i]['label'][-1:])+1)

100%|██████████| 6291252/6291252 [02:12<00:00, 47626.49it/s]


In [12]:
final_data_path = f"s3://sagemaker-studio-ai-lab-3/samples/datasets/bert60k_predictions"
reviews.to_csv(final_data_path)

## Visualizing results and generating score

In [9]:
subset = reviews[['r_text', 'r_stars', 'bert_pred']]
subset.head(5)

,r_text,r_stars,bert_pred
0,It's a basic speakeasy bar that's more low key...,4,"{'label': 'LABEL_3', 'score': 0.5951117873191833}"
1,I really enjoy this mall. Its pretty much Del...,4,"{'label': 'LABEL_4', 'score': 0.8126952648162842}"
2,Just moved to the area and was really excited ...,4,"{'label': 'LABEL_4', 'score': 0.9655696153640747}"
3,Milktooth was such a unique experience. I had ...,5,"{'label': 'LABEL_5', 'score': 0.6034464240074158}"
4,Wide variety of micro brews with great traditi...,4,"{'label': 'LABEL_4', 'score': 0.9898878931999207}"


In [10]:
range_miss = []
for i in tqdm(range(len(subset))):
    if subset.r_stars[i] != int(subset.bert_pred[i]['label'][-1:]):
        range_miss.append(abs(int(subset.bert_pred[i]['label'][-1:]) - subset.r_stars[i]))

100%|██████████| 6291252/6291252 [01:50<00:00, 56745.26it/s]


In [11]:
print(f'Number of wrongly predicted rows: {len(range_miss)}/{len(subset)}')
print(f'Off by 1: {range_miss.count(1)}')
print(f'Off by 2: {range_miss.count(2)}')
print(f'Off by 3: {range_miss.count(3)}')
print(f'Off by 4: {range_miss.count(4)}')
print(f'Off by 5: {range_miss.count(5)}')

Number of wrongly predicted rows: 1678396/6291252
Off by 1: 1553395
Off by 2: 97678
Off by 3: 13137
Off by 4: 14186
Off by 5: 0


bert-base-45k-128maxlen
Number of wrongly predicted rows: 1957788/6291252
Off by 1: 1723499
Off by 2: 178163
Off by 3: 26816
Off by 4: 29310
Off by 5: 0

bert-base-60k-512maxlen
Number of wrongly predicted rows: 1678396/6291252
Off by 1: 1553395
Off by 2: 97678
Off by 3: 13137
Off by 4: 14186
Off by 5: 0